In [8]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

driver = webdriver.Chrome('D:\webcrawling\chromedriver.exe')

<ipython-input-8-78626fa46154>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('D:\webcrawling\chromedriver.exe')


In [9]:
url_format = "https://www.genie.co.kr/genre/{}?genreCode={}&pg={}"
genreCode = ["M0100"]

songID = []
albumID = []

for gCode in genreCode:
    for page in range(1,2):
        url = url_format.format(gCode, gCode, page) #크롤링할 URL
        driver.get(url)
        chart = driver.page_source
        soup = BeautifulSoup(chart, 'html.parser') #URL의 HTML을 읽어옴

        # 해당 HTML에서 selector 영역만 parsing 
        selector = "#body-content > div.songlist-box > div > table > tbody > tr.list"
        raw_data = soup.select(selector)
        for tag in raw_data:
            songID.append( tag.get('songid') )

        # 해당 HTML에서 selector 영역만 parsing (앨범 정보 영역)
        selector = "#body-content > div.songlist-box > div > table > tbody > tr.list > td.info > a.albumtitle.ellipsis"
        raw_data = soup.select(selector)
        for tag in raw_data:
            string = tag.get('onclick')
            album_id = re.sub(r'[^0-9]', '', string)
            albumID.append( album_id )


In [13]:
songURL_format = "https://www.genie.co.kr/detail/songInfo?xgnm={}"
albumURL_format = "https://www.genie.co.kr/detail/albumInfo?axnm={}"

list_selector = ["#body-content > div.song-main-infos > div.info-zone > h2",
                 "#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(1) > span.value > a",
                 "#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(6) > span.value",
                 "#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(5) > span.value",
                 "#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(7) > span.value",
                 "#body-content > div.song-main-infos > div.aside-zone.daily-chart > div.total > div:nth-child(2) > p",
                 "#emLikeCount",
                 "#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(3) > span.value",
                 "#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value",
                ]

list_selector_album = ["#body-content > div.album-detail-infos > div.info-zone > ul > li:nth-child(5) > span.value",
                       "#body-content > div.album-detail-infos > div.info-zone > ul > li:nth-child(4) > span.value",
                       "#body-content > div.album-detail-infos > div.info-zone > ul > li:nth-child(3) > span.value",
                      ]

songInfo = [] #곡의 정보를 list형태로 저장하기 위한 list
albumInfo = [] # (앨범)데이터 임시 저장소, 나중에 songInfo와 합병시킴


# song id로부터 곡의 정보들을 얻는다
url = songURL_format.format("98769383")
driver.get(url)
chart = driver.page_source
soup = BeautifulSoup(chart, 'html.parser')

selector = "#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value"
raw_data = soup.select(selector)

li = []

for tag in raw_data:
    li.append( tag.get_text().strip() )

print(li)

    

 
        
        # tmpList.append( tag.get_text().strip() )
  


    


['']


In [11]:
# 곡 명, 스트리밍 수, 좋아요 수 --> ,를 ''로 바꾸어 주자
# 가수, 작곡가, 작사가, 편곡자, 기획사, 발매사 --> ,를 & 로 바꾸어 주자.


f = open("./test.csv", "w")

tup = "곡 명, 가수, 작곡가, 작사가, 편곡자, 스트리밍 수, 좋아요 수, 발매시점, 기획사, 발매사, 장르, 재생시간\n"
f.write(tup)

for row in songInfo:
    index = 1
    for col in row:
        if index in (1, 6, 7):
            col = col.replace(',', '') 
        elif index in (2, 3, 4, 5, 9, 10):
            col = col.replace(',', ' & ')
        
        if index == 12:
            col += '\n'
        else:
            col += ", "
            
        f.write(col)
        index += 1
        
f.close()